In [8]:
import pandas as pd
import os

first_csv = pd.read_csv('Chile_all(1).csv', sep=';')

In [9]:
second_csv = pd.DataFrame()
size = len(os.listdir('./data'))

for i in range(1, size + 1):
    df = pd.read_csv(f'./data/Chile_all1_part{i}.csv', sep=';')
    second_csv = pd.concat([second_csv, df], ignore_index=True)


In [ ]:
def read_tripadvisor_data(path: str) -> pd.DataFrame:
    df = pd.DataFrame()
    size = len(os.listdir(path))

    for i in range(size):
        file_path = os.path.join(path, f"Chile_all1_part{i}.csv")
        df = pd.concat([df, pd.read_csv(file_path, sep=';')], ignore_index=True)

    return df